<a href="https://colab.research.google.com/github/psgrigoletti/margem-liquida-colab/blob/main/2022_05_Stock_Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalações e importações

In [ ]:
!pip install yfinance
!pip install ta

In [118]:
import pandas as pd
import numpy as np
import pytz
import plotly.graph_objects as go
import yfinance as yf
import ta as ta
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import ssl
import random

# Fazendo web scraping do índice IBOV

In [71]:
def busca_carteira_teorica(indice):
  ssl._create_default_https_context = ssl._create_unverified_context
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return pd.read_html(url, decimal=',', thousands='.', index_col='Código')[0][:-1]

In [116]:
indice_selecionado = "ibov" #@param ["ifix", "ibov"]
indice = busca_carteira_teorica(indice_selecionado)

In [117]:
ativos_indice = indice.index.to_list()
tickers = [ticker + '.SA' for ticker in ativos_indice]
tickers.sort()
print(tickers)

['ABEV3.SA', 'ASAI3.SA', 'AZUL4.SA', 'B3SA3.SA', 'BBAS3.SA', 'BBDC3.SA', 'BBDC4.SA', 'BBSE3.SA', 'BEEF3.SA', 'BPAC11.SA', 'BRAP4.SA', 'BRDT3.SA', 'BRFS3.SA', 'BRKM5.SA', 'BRML3.SA', 'BTOW3.SA', 'CCRO3.SA', 'CIEL3.SA', 'CMIG4.SA', 'COGN3.SA', 'CPFE3.SA', 'CPLE6.SA', 'CRFB3.SA', 'CSAN3.SA', 'CSNA3.SA', 'CVCB3.SA', 'CYRE3.SA', 'ECOR3.SA', 'EGIE3.SA', 'ELET3.SA', 'ELET6.SA', 'EMBR3.SA', 'ENBR3.SA', 'ENEV3.SA', 'ENGI11.SA', 'EQTL3.SA', 'EZTC3.SA', 'FLRY3.SA', 'GGBR4.SA', 'GNDI3.SA', 'GOAU4.SA', 'GOLL4.SA', 'HAPV3.SA', 'HGTX3.SA', 'HYPE3.SA', 'IGTA3.SA', 'IRBR3.SA', 'ITSA4.SA', 'ITUB4.SA', 'JBSS3.SA', 'JHSF3.SA', 'KLBN11.SA', 'LAME4.SA', 'LCAM3.SA', 'LREN3.SA', 'MGLU3.SA', 'MRFG3.SA', 'MRVE3.SA', 'MULT3.SA', 'NTCO3.SA', 'PCAR3.SA', 'PETR3.SA', 'PETR4.SA', 'PRIO3.SA', 'QUAL3.SA', 'RADL3.SA', 'RAIL3.SA', 'RENT3.SA', 'SANB11.SA', 'SBSP3.SA', 'SULA11.SA', 'SUZB3.SA', 'TAEE11.SA', 'TIMS3.SA', 'TOTS3.SA', 'UGPA3.SA', 'USIM5.SA', 'VALE3.SA', 'VIVT3.SA', 'VVAR3.SA', 'WEGE3.SA', 'YDUQ3.SA']


# Buscando dados

In [89]:
data_fim = datetime.now(pytz.timezone("America/Sao_Paulo"))
data_fim = data_fim.strftime('%Y-%m-%d')
data_fim

'2022-05-22'

In [90]:
data_inicio = datetime.now(pytz.timezone("America/Sao_Paulo")) - timedelta(days=100)
data_inicio = data_inicio.strftime('%Y-%m-%d')
data_inicio

'2022-02-11'

In [92]:
lista_OHLC = []
indice_OHLC = []

for i in tickers:
  ssl._create_default_https_context = ssl.create_default_context()
  cotacoes = yf.download([i], start=data_inicio, end=data_fim, interval = "1h", progress=False, show_errors=False)
  if(len(cotacoes)>0):
      cotacoes['Ativo'] = i
      indice_OHLC.append(i)
      lista_OHLC.append(cotacoes)
  else:
      print(i + " não tem cotação completa no período. Serão ignorados...")

BRDT3.SA não tem cotação completa no período. Serão ignorados...
BTOW3.SA não tem cotação completa no período. Serão ignorados...
GNDI3.SA não tem cotação completa no período. Serão ignorados...
HGTX3.SA não tem cotação completa no período. Serão ignorados...
IGTA3.SA não tem cotação completa no período. Serão ignorados...
LAME4.SA não tem cotação completa no período. Serão ignorados...
VVAR3.SA não tem cotação completa no período. Serão ignorados...


In [93]:
for i in range(0,len(lista_OHLC)):
  MM9p = ta.trend.SMAIndicator(lista_OHLC[i]['Close'], window=9)
  MM20p = ta.trend.SMAIndicator(lista_OHLC[i]['Close'], window=20)
  MM200p = ta.trend.SMAIndicator(lista_OHLC[i]['Close'], window=200)
  IFR14p = ta.momentum.RSIIndicator(close=lista_OHLC[i]['Close'], window=14)
  lista_OHLC[i]['SMA_9p'] = MM20p.sma_indicator()
  lista_OHLC[i]['SMA_20p'] = MM20p.sma_indicator()
  lista_OHLC[i]['SMA_200p'] = MM200p.sma_indicator()
  lista_OHLC[i]['IFR14p'] = IFR14p.rsi()

In [114]:
print(indice_OHLC)

['ABEV3.SA', 'ASAI3.SA', 'AZUL4.SA', 'B3SA3.SA', 'BBAS3.SA', 'BBDC3.SA', 'BBDC4.SA', 'BBSE3.SA', 'BEEF3.SA', 'BPAC11.SA', 'BRAP4.SA', 'BRFS3.SA', 'BRKM5.SA', 'BRML3.SA', 'CCRO3.SA', 'CIEL3.SA', 'CMIG4.SA', 'COGN3.SA', 'CPFE3.SA', 'CPLE6.SA', 'CRFB3.SA', 'CSAN3.SA', 'CSNA3.SA', 'CVCB3.SA', 'CYRE3.SA', 'ECOR3.SA', 'EGIE3.SA', 'ELET3.SA', 'ELET6.SA', 'EMBR3.SA', 'ENBR3.SA', 'ENEV3.SA', 'ENGI11.SA', 'EQTL3.SA', 'EZTC3.SA', 'FLRY3.SA', 'GGBR4.SA', 'GOAU4.SA', 'GOLL4.SA', 'HAPV3.SA', 'HYPE3.SA', 'IRBR3.SA', 'ITSA4.SA', 'ITUB4.SA', 'JBSS3.SA', 'JHSF3.SA', 'KLBN11.SA', 'LCAM3.SA', 'LREN3.SA', 'MGLU3.SA', 'MRFG3.SA', 'MRVE3.SA', 'MULT3.SA', 'NTCO3.SA', 'PCAR3.SA', 'PETR3.SA', 'PETR4.SA', 'PRIO3.SA', 'QUAL3.SA', 'RADL3.SA', 'RAIL3.SA', 'RENT3.SA', 'SANB11.SA', 'SBSP3.SA', 'SULA11.SA', 'SUZB3.SA', 'TAEE11.SA', 'TIMS3.SA', 'TOTS3.SA', 'UGPA3.SA', 'USIM5.SA', 'VALE3.SA', 'VIVT3.SA', 'WEGE3.SA', 'YDUQ3.SA']


In [119]:
ticker_selecionado = random.choice(indice_OHLC)
indice_ticker_selecionado = indice_OHLC.index(ticker_selecionado)
indice_ticker_selecionado

30

In [122]:
# Como exemplo, vamos produzir os gráficos de candlestick (com suas MMs), OBV e IFR do ticker selecionado
df_figure = lista_OHLC[indice_ticker_selecionado]
df_figure

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Candlestick(name=ticker_selecionado, x=df_figure.index, open=df_figure['Open'], high = df_figure['High'], low=df_figure['Low'], close=df_figure['Close']), row=1,col=1)
fig.add_trace(go.Scatter(name='SMA_9p', x=df_figure.index, y=df_figure['SMA_9p'],marker_color='red'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_20p', x=df_figure.index, y=df_figure['SMA_20p'],marker_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_200p', x=df_figure.index, y=df_figure['SMA_200p'],marker_color='black'), row=1, col=1)
fig.add_trace(go.Scatter(name='IFR14p', x=df_figure.index, y=df_figure['IFR14p'],marker_color='orange'), row=2, col=1)
fig.update_xaxes(title_text="<b> Data",rangebreaks=[dict(bounds=["sat", "mon"]),dict(bounds=[17, 10], pattern="hour")])
fig.update_layout(xaxis_rangeslider_visible=False, title_text='Price, OBV e IFR',width=1000,height=1000)
go.Figure(fig.show())

# 3. Screening dos ativos listados
Vamos determinar algumas regras e fazer a varredura de quantos e quais ativos se encaixam nesse momento. Estaremos buscando ativos que apresentaram:

- **Regra 01:** Fechamento com preço acima da MM20p e MM200p;

- **Regra 02:** MM9p > MM20p > MM200p;

- **Regra 03:** IFR14p < 50

In [108]:
lista_screening = []

for i in range(0,len(lista_OHLC)):
    passou_na_regra = lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_20p'] and \
                      lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_200p'] and \
                      lista_OHLC[i].iloc[-1]['SMA_9p'] >= lista_OHLC[i].iloc[-1]['SMA_20p'] and \
                      lista_OHLC[i].iloc[-1]['SMA_20p'] >= lista_OHLC[i].iloc[-1]['SMA_200p'] and \
                      lista_OHLC[i].iloc[-1]['IFR14p'] < 50
    if passou_na_regra:
        lista_screening.append((tickers[i],i))
        print(tickers[i] + " passou nas regras.")

#lista_screening

ITUB4.SA passou nas regras.
PETR3.SA passou nas regras.


In [111]:
lista_screening = []

for i in range(0,len(lista_OHLC)):
    passou_na_regra = lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_20p'] and \
                      lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_200p'] and \
                      lista_OHLC[i].iloc[-1]['SMA_9p'] >= lista_OHLC[i].iloc[-1]['SMA_20p'] and \
                      lista_OHLC[i].iloc[-1]['SMA_20p'] >= lista_OHLC[i].iloc[-1]['SMA_200p'] and \
                      lista_OHLC[i].iloc[-1]['IFR14p'] < 50
    if passou_na_regra:
        lista_screening.append((tickers[i],i))
        print("O ativo indexado como",i,"("+tickers[i]+") atende a todas as regras estabelecidas")

#lista_screening

O ativo indexado como 48 (ITUB4.SA) atende a todas as regras estabelecidas
O ativo indexado como 61 (PETR3.SA) atende a todas as regras estabelecidas


In [113]:
# Checagem gráfica de um ativo que atende a todas as regras do Screening de Compra
# Cheque todas as regras nos gráficos e veja que realmente funcionou
indice = 48

df_figure = lista_OHLC[indice]
df_figure

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Candlestick(name=tickers[indice], x=df_figure.index, open=df_figure['Open'], high = df_figure['High'], low=df_figure['Low'], close=df_figure['Close']), row=1,col=1)
fig.add_trace(go.Scatter(name='SMA_9p', x=df_figure.index, y=df_figure['SMA_9p'],marker_color='red'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_20p', x=df_figure.index, y=df_figure['SMA_20p'],marker_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_200p', x=df_figure.index, y=df_figure['SMA_200p'],marker_color='black'), row=1, col=1)
fig.add_trace(go.Scatter(name='IFR14p', x=df_figure.index, y=df_figure['IFR14p'],marker_color='orange'), row=2, col=1)
fig.update_xaxes(title_text="<b> Data",rangebreaks=[dict(bounds=["sat", "mon"]),dict(bounds=[17, 10], pattern="hour")])
fig.update_layout(xaxis_rangeslider_visible=False, title_text='Price, OBV e IFR',width=1000,height=1000)
fig.show()